# Volcanoes on Venus — End-to-End Notebook

This notebook reproduces the full pipeline on the JARtool Magellan SAR dataset:
- Data extraction into chips (optional if already prepared)
- Train/Val/Test splits with stratification
- Custom CNN training and evaluation
- Optimizer comparison (SGD, SGD+Momentum)
- Transfer learning with ResNet18 and VGG16

## 1. Setup and configuration

In [2]:
# Paths and switches
DATA_ROOT = "data_processed"          # expected final dataset layout
INTERMEDIATE_ROOT = "data_intermediate"
RAW_PACKAGE_DIR = "package"           # where the extracted UCI tar contents live
RESULTS_DIR = "results"

# Fast mode: skip raw extraction if you already built data_processed/
FAST_MODE = True

# Training configs
IMG_SIZE_CUSTOM = 32      # for custom CNN
IMG_SIZE_PRETRAIN = 224   # for ResNet/VGG
BATCH_CUSTOM = 128
BATCH_RESNET = 64
BATCH_VGG = 32
EPOCHS_CUSTOM = 20
EPOCHS_PRETRAIN = 12

# Reproducibility
SEED = 42

print("Config loaded.")

Config loaded.


## 2. Imports

In [3]:
import os, math, random
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## 3. Optional raw reading and chip extraction

If you already have `data_processed/` prepared, keep `FAST_MODE = True` and skip this section.
Otherwise, set `FAST_MODE = False` and run these cells to:
- Read `.spr/.sdt` files using Python `vread`
- Load experiment chip files in VIEW format
- Build `data_intermediate/` arrays and a stratified split under `data_processed/`

In [4]:
import struct

def vread_py(basename):
    '''
    Python equivalent of vread.m for VIEW format.
    Reads `${basename}.spr` to parse metadata, then reads `${basename}.sdt`.
    Returns a numpy array with shape (nr, nc) for 2D arrays.
    '''
    spr = f"{basename}.spr"
    sdt = f"{basename}.sdt"

    with open(spr, "r") as f:
        # spr layout:
        # ndim, nc, junk, junk, nr, junk, junk, type
        tokens = f.read().split()
        ndim = int(tokens[0])
        if ndim != 2:
            raise ValueError("Only 2D data supported")
        nc = int(tokens[1])
        # tokens[2], tokens[3] are junk floats
        nr = int(tokens[4])
        # tokens[5], tokens[6] are junk floats
        dtype_code = int(tokens[7])

    if dtype_code == 0:
        dtype = np.uint8
    elif dtype_code == 2:
        dtype = np.int32
    elif dtype_code == 3:
        dtype = np.float32
    elif dtype_code == 5:
        dtype = np.float64
    else:
        raise ValueError("Unrecognized data type")

    with open(sdt, "rb") as f:
        raw = f.read()
    arr = np.frombuffer(raw, dtype=dtype, count=nr*nc)
    arr = arr.reshape((nr, nc))
    return arr

print("vread_py ready. Set FAST_MODE=False to use it.")

vread_py ready. Set FAST_MODE=False to use it.


## 4. Split builder

In [5]:
from collections import Counter
from PIL import Image

def ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

def stratified_split(indices, labels, train_frac=0.7, val_frac=0.15, test_frac=0.15, seed=SEED):
    rng = np.random.default_rng(seed)
    idx_by_class = {}
    for i, y in zip(indices, labels):
        idx_by_class.setdefault(int(y), []).append(i)
    splits = {"train": [], "val": [], "test": []}
    for cls, arr in idx_by_class.items():
        rng.shuffle(arr)
        n = len(arr)
        n_train = int(round(train_frac * n))
        n_val = int(round(val_frac * n))
        train_idx = arr[:n_train]
        val_idx = arr[n_train:n_train+n_val]
        test_idx = arr[n_train+n_val:]
        splits["train"].extend(train_idx)
        splits["val"].extend(val_idx)
        splits["test"].extend(test_idx)
    return splits

def save_png_chips(chips_hw, labels, splits, out_root="data_processed", img_size=32):
    '''
    chips_hw: (N, H, W) uint8
    labels: (N,) 0=non_volcano, 1=volcano
    '''
    ensure_dir(out_root)
    for split in ["train","val","test"]:
        for cname in ["non_volcano","volcano"]:
            ensure_dir(Path(out_root)/split/cname)

    for split_name, idx_list in splits.items():
        for idx in idx_list:
            lab = int(labels[idx])
            cname = "volcano" if lab==1 else "non_volcano"
            arr = chips_hw[idx]
            img = Image.fromarray(arr, mode="L").resize((img_size, img_size), Image.NEAREST)
            fname = f"chip_{idx:06d}.png"
            img.save(Path(out_root)/split_name/cname/fname)

    for split in ["train","val","test"]:
        n0 = len(list(Path(out_root, split, "non_volcano").glob("*.png")))
        n1 = len(list(Path(out_root, split, "volcano").glob("*.png")))
        print(f"{split}: non_volcano={n0}, volcano={n1}")

print("Split utilities ready.")

Split utilities ready.


## 5. Build or reuse splits

In [6]:
if FAST_MODE and Path(DATA_ROOT).exists():
    print("FAST_MODE=True and data_processed/ exists. Skipping extraction.")
else:
    chips_path = Path(INTERMEDIATE_ROOT, "chips_all.npy")
    labels_path = Path(INTERMEDIATE_ROOT, "labels_all.npy")
    if chips_path.exists() and labels_path.exists():
        chips_hw = np.load(chips_path)    # (N, 15, 15), uint8
        labels = np.load(labels_path)     # (N,), 0 or 1
        N = chips_hw.shape[0]
        idx = np.arange(N)
        splits = stratified_split(idx, labels, 0.7, 0.15, 0.15, seed=SEED)
        save_png_chips(chips_hw, labels, splits, out_root=DATA_ROOT, img_size=32)
    else:
        raise SystemExit("No precomputed chips_all.npy and labels_all.npy found. "
                         "Set FAST_MODE=True to reuse existing data_processed/ or prepare intermediates.")

FAST_MODE=True and data_processed/ exists. Skipping extraction.


## 6. PyTorch dataloaders

In [7]:
def get_transforms(img_size=32):
    train_tf = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])
    eval_tf = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])
    return train_tf, eval_tf

def _make_weighted_sampler(image_folder_dataset):
    targets = [cls for (_, cls) in image_folder_dataset.samples]
    targets = np.array(targets)
    class_counts = np.bincount(targets)
    class_weights = 1.0 / class_counts
    sample_weights = class_weights[targets]
    sampler = WeightedRandomSampler(
        weights=torch.as_tensor(sample_weights, dtype=torch.double),
        num_samples=len(sample_weights),
        replacement=True
    )
    return sampler, class_counts

def get_dataloaders(data_root=DATA_ROOT, img_size=32, batch_size=128, num_workers=2, use_weighted_sampler=True):
    train_tf, eval_tf = get_transforms(img_size)
    train_ds = datasets.ImageFolder(f"{data_root}/train", transform=train_tf)
    val_ds   = datasets.ImageFolder(f"{data_root}/val",   transform=eval_tf)
    test_ds  = datasets.ImageFolder(f"{data_root}/test",  transform=eval_tf)

    if use_weighted_sampler:
        sampler, class_counts = _make_weighted_sampler(train_ds)
        train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
    else:
        class_counts = None
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader, test_loader, train_ds.class_to_idx, class_counts

print("Dataloaders ready.")

Dataloaders ready.


## 7. Custom CNN model

In [8]:
class VolcanoCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 128), nn.ReLU(inplace=True), nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

print("VolcanoCNN defined.")

VolcanoCNN defined.


## 8. Training helpers

In [9]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_seen = 0.0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.item()) * imgs.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += int((preds == labels).sum().item())
        total_seen += labels.size(0)
    return total_loss/total_seen, (total_correct/total_seen if total_seen else 0.0)

@torch.no_grad()
def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_seen = 0.0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)
        total_loss += float(loss.item()) * imgs.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += int((preds == labels).sum().item())
        total_seen += labels.size(0)
    return total_loss/total_seen, (total_correct/total_seen if total_seen else 0.0)

def plot_curve(history, title, out_png):
    plt.figure()
    plt.plot(history["epoch"], history["train_loss"], label="train_loss")
    plt.plot(history["epoch"], history["val_loss"], label="val_loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(title)
    plt.legend(); plt.tight_layout()
    Path(out_png).parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_png, dpi=200); plt.close()

## 9. Train custom CNN (Adam, 20 epochs)

In [ ]:
RESULTS_DIR = "results"
Path(RESULTS_DIR).mkdir(exist_ok=True)

train_loader, val_loader, test_loader, class_to_idx, class_counts = get_dataloaders(
    data_root=DATA_ROOT, img_size=IMG_SIZE_CUSTOM, batch_size=BATCH_CUSTOM, num_workers=2, use_weighted_sampler=True
)
print("class_to_idx:", class_to_idx)
print("class_counts (train):", class_counts)

model = VolcanoCNN(num_classes=2).to(device)
if class_counts is not None:
    non_v, vol = float(class_counts[0]), float(class_counts[1])
    weights = torch.tensor([1.0, non_v/vol], dtype=torch.float32, device=device)
else:
    weights = None
criterion = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
history = {"epoch": [], "train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
best_val_acc = 0.0
best_path = Path(RESULTS_DIR)/"custom_cnn_adam"/"best_model.pth"
best_path.parent.mkdir(parents=True, exist_ok=True)

for epoch in range(1, EPOCHS_CUSTOM+1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)
    print(f"Epoch {epoch:02d} | train_loss={tr_loss:.4f} val_loss={val_loss:.4f} | train_acc={tr_acc:.4f} val_acc={val_acc:.4f}")
    history["epoch"].append(epoch)
    history["train_loss"].append(tr_loss); history["val_loss"].append(val_loss)
    history["train_acc"].append(tr_acc);   history["val_acc"].append(val_acc)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_path)

plot_curve(history, "Custom CNN training", str(Path(RESULTS_DIR)/"figs"/"custom_loss_curve_adam.png"))
print("Best model saved to:", best_path)

class_to_idx: {'non_volcano': 0, 'volcano': 1}
class_counts (train): [92447  4305]


KeyboardInterrupt: 

## 10. Test evaluation for custom CNN

In [10]:
best_path = Path(RESULTS_DIR)/"custom_cnn"/"best_model.pth"
model = VolcanoCNN(num_classes=2).to(device)
model.load_state_dict(torch.load(best_path, map_location=device))

preds_all, labels_all = [], []
model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        preds = torch.argmax(logits, dim=1)
        preds_all.append(preds.cpu().numpy())
        labels_all.append(labels.cpu().numpy())

preds_all = np.concatenate(preds_all)
labels_all = np.concatenate(labels_all)

acc = (preds_all == labels_all).mean()
prec = precision_score(labels_all, preds_all, pos_label=1, zero_division=0)
rec  = recall_score(labels_all, preds_all, pos_label=1, zero_division=0)
cm   = confusion_matrix(labels_all, preds_all, labels=[0,1])
acc, prec, rec, cm

NameError: name 'test_loader' is not defined

In [ ]:
def plot_cm(cm, class_names, out_path):
    fig, ax = plt.subplots(figsize=(4,4))
    im = ax.imshow(cm, cmap="Blues")
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(len(class_names)), yticks=np.arange(len(class_names)),
           xticklabels=class_names, yticklabels=class_names,
           xlabel="Predicted", ylabel="True", title="Confusion Matrix (Test)")
    thresh = cm.max()/2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i,j], "d"),
                    ha="center", va="center",
                    color="white" if cm[i,j] > thresh else "black")
    fig.tight_layout()
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path, dpi=200); plt.close()

idx_to_class = {v:k for k,v in class_to_idx.items()}
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]
plot_cm(cm, class_names, str(Path(RESULTS_DIR)/"figs"/"confusion_matrix_custom.png"))
print("Saved confusion matrix to results/figs/confusion_matrix_custom.png")

## 11. Optimizer comparison (SGD, SGD+Momentum)

In [ ]:
def run_with_optimizer(optimizer_builder, tag):
    train_loader, val_loader, _, _, class_counts = get_dataloaders(
        data_root=DATA_ROOT, img_size=IMG_SIZE_CUSTOM, batch_size=BATCH_CUSTOM, num_workers=2, use_weighted_sampler=True
    )
    model = VolcanoCNN(num_classes=2).to(device)
    if class_counts is not None:
        non_v, vol = float(class_counts[0]), float(class_counts[1])
        weights = torch.tensor([1.0, non_v/vol], dtype=torch.float32, device=device)
    else:
        weights = None
    criterion = nn.CrossEntropyLoss(weight=weights)
    optimizer = optimizer_builder(model)

    history = {"epoch": [], "train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    best_val_acc = 0.0
    best_path = Path(RESULTS_DIR)/f"custom_cnn_{tag}"/"best_model.pth"
    best_path.parent.mkdir(parents=True, exist_ok=True)

    for epoch in range(1, EPOCHS_CUSTOM+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)
        print(f"[{tag}] Epoch {epoch:02d} | train_loss={tr_loss:.4f} val_loss={val_loss:.4f} | train_acc={tr_acc:.4f} val_acc={val_acc:.4f}")
        history["epoch"].append(epoch)
        history["train_loss"].append(tr_loss); history["val_loss"].append(val_loss)
        history["train_acc"].append(tr_acc);   history["val_acc"].append(val_acc)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_path)

    plot_curve(history, f"Custom CNN training ({tag})", str(Path(RESULTS_DIR)/"figs"/f"custom_loss_curve_{tag}.png"))
    print(f"[{tag}] Best model saved to:", best_path)

# SGD and SGD+Momentum
run_with_optimizer(lambda m: optim.SGD(m.parameters(), lr=0.01), "sgd")
run_with_optimizer(lambda m: optim.SGD(m.parameters(), lr=0.01, momentum=0.9), "sgdm")

## 12. Transfer learning: ResNet18 and VGG16

In [ ]:
def replace_conv1_with_grayscale_resnet(resnet):
    old = resnet.conv1
    new = nn.Conv2d(1, old.out_channels, kernel_size=old.kernel_size, stride=old.stride, padding=old.padding, bias=False)
    with torch.no_grad():
        new.weight.copy_(old.weight.mean(dim=1, keepdim=True))
    resnet.conv1 = new
    return resnet

def replace_first_conv_vgg16(vgg):
    first = vgg.features[0]
    new = nn.Conv2d(1, first.out_channels, kernel_size=first.kernel_size, stride=first.stride, padding=first.padding, bias=first.bias is not None)
    with torch.no_grad():
        new.weight.copy_(first.weight.mean(dim=1, keepdim=True))
        if first.bias is not None:
            new.bias.copy_(first.bias)
    vgg.features[0] = new
    return vgg

def train_backbone(model, tag, img_size, batch_size, epochs=EPOCHS_PRETRAIN, lr=1e-4):
    train_loader, val_loader, test_loader, class_to_idx, class_counts = get_dataloaders(
        data_root=DATA_ROOT, img_size=img_size, batch_size=batch_size, num_workers=2, use_weighted_sampler=True
    )
    if class_counts is not None:
        non_v, vol = float(class_counts[0]), float(class_counts[1])
        weights = torch.tensor([1.0, non_v/vol], dtype=torch.float32, device=device)
    else:
        weights = None
    criterion = nn.CrossEntropyLoss(weight=weights)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    history = {"epoch": [], "train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    best_val_acc = 0.0
    best_path = Path(RESULTS_DIR)/tag/"best_model.pth"
    best_path.parent.mkdir(parents=True, exist_ok=True)

    for epoch in range(1, epochs+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)
        print(f"[{tag}] Epoch {epoch:02d} | train_loss={tr_loss:.4f} val_loss={val_loss:.4f} | train_acc={tr_acc:.4f} val_acc={val_acc:.4f}")
        history["epoch"].append(epoch)
        history["train_loss"].append(tr_loss); history["val_loss"].append(val_loss)
        history["train_acc"].append(tr_acc);   history["val_acc"].append(val_acc)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_path)

    plot_curve(history, f"{tag} fine-tuning", str(Path(RESULTS_DIR)/"figs"/f"custom_loss_curve_{tag}.png"))
    print(f"[{tag}] Best model saved to:", best_path)

    model.load_state_dict(torch.load(best_path, map_location=device))
    preds_all, labels_all = [], []
    model.eval()
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            preds = torch.argmax(logits, dim=1)
            preds_all.append(preds.cpu().numpy())
            labels_all.append(labels.cpu().numpy())
    preds_all = np.concatenate(preds_all)
    labels_all = np.concatenate(labels_all)

    acc = (preds_all == labels_all).mean()
    prec = precision_score(labels_all, preds_all, pos_label=1, zero_division=0)
    rec  = recall_score(labels_all, preds_all, pos_label=1, zero_division=0)

    from sklearn.metrics import confusion_matrix
    cm   = confusion_matrix(labels_all, preds_all, labels=[0,1])

    idx_to_class = {v:k for k,v in class_to_idx.items()}
    class_names = [idx_to_class[i] for i in range(len(idx_to_class))]
    def _plot_cm(cm, class_names, out_path):
        fig, ax = plt.subplots(figsize=(4,4))
        im = ax.imshow(cm, cmap="Blues")
        ax.figure.colorbar(im, ax=ax)
        ax.set(xticks=np.arange(len(class_names)), yticks=np.arange(len(class_names)),
               xticklabels=class_names, yticklabels=class_names,
               xlabel="Predicted", ylabel="True", title="Confusion Matrix (Test)")
        thresh = cm.max()/2
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i,j], "d"),
                        ha="center", va="center",
                        color="white" if cm[i,j] > thresh else "black")
        fig.tight_layout()
        Path(out_path).parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(out_path, dpi=200); plt.close()
    _plot_cm(cm, class_names, str(Path(RESULTS_DIR)/"figs"/f"confusion_matrix_{tag}.png"))
    print(f"[{tag}] Test: acc={acc:.4f} precision={prec:.4f} recall={rec:.4f}")
    return acc, prec, rec

# ResNet18
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet18 = replace_conv1_with_grayscale_resnet(resnet18)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 2)
resnet18 = resnet18.to(device)
acc_res, prec_res, rec_res = train_backbone(resnet18, tag="resnet18", img_size=IMG_SIZE_PRETRAIN, batch_size=BATCH_RESNET)

# VGG16
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16 = replace_first_conv_vgg16(vgg16)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 2)
vgg16 = vgg16.to(device)
acc_vgg, prec_vgg, rec_vgg = train_backbone(vgg16, tag="vgg16", img_size=IMG_SIZE_PRETRAIN, batch_size=BATCH_VGG)

## 13. Final comparison table

In [ ]:
import pandas as pd

best_path_cnn = Path(RESULTS_DIR)/"custom_cnn"/"best_model.pth"
cnn = VolcanoCNN(num_classes=2).to(device)
cnn.load_state_dict(torch.load(best_path_cnn, map_location=device))
preds_all, labels_all = [], []
cnn.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = cnn(imgs)
        preds = torch.argmax(logits, dim=1)
        preds_all.append(preds.cpu().numpy())
        labels_all.append(labels.cpu().numpy())
preds_all = np.concatenate(preds_all)
labels_all = np.concatenate(labels_all)
acc_cnn  = (preds_all == labels_all).mean()
prec_cnn = precision_score(labels_all, preds_all, pos_label=1, zero_division=0)
rec_cnn  = recall_score(labels_all, preds_all, pos_label=1, zero_division=0)

df = pd.DataFrame([
    {"Model": "Custom CNN (Adam)", "Test Accuracy": acc_cnn, "Precision (volc)": prec_cnn, "Recall (volc)": rec_cnn},
    {"Model": "ResNet18 (FT)",     "Test Accuracy": acc_res, "Precision (volc)": prec_res, "Recall (volc)": rec_res},
    {"Model": "VGG16 (FT)",        "Test Accuracy": acc_vgg, "Precision (volc)": prec_vgg, "Recall (volc)": rec_vgg},
])

import caas_jupyter_tools
caas_jupyter_tools.display_dataframe_to_user("Final Comparison", df)

df